# Linear Models and Classifiers

This notebook reproduces the results from the initial paper, following the given information as closely as possible.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
df = pd.read_csv("Data/rebuild_features.csv")

In [ ]:
df.columns.values

In [ ]:
df.describe()

# Preprocessing and feature selection

In [ ]:
numBat1 = len([i for i in list(df.cell_key) if i[1] == "1"])
numBat2 = len([i for i in list(df.cell_key) if i[1] == "2"])
numBat3 = len([i for i in list(df.cell_key) if i[1] == "3"])
numBat = sum((numBat1,numBat2,numBat3))

In [ ]:
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat);

splits = [train_ind, test_ind, secondary_test_ind]

In [ ]:
# Define feature and target columns for regression models

varmod_features = ["variance_dQ_100_10"]
dismod_features = [
    "variance_dQ_100_10",
    "minimum_dQ_100_10",
    "skewness_dQ_100_10",
    "kurtosis_dQ_100_10",
    "discharge_capacity_2",
    "diff_discharge_capacity_max_2",
]
fullmod_features = [
    "minimum_dQ_100_10",
    "variance_dQ_100_10",
    "slope_lin_fit_2_100",
    "intercept_lin_fit_2_100",
    "discharge_capacity_2",
    "mean_charge_time_2_6",
    "minimum_IR_2_100",
    "diff_IR_100_2",
]
targetmod = ["cycle_life"]

# Define feature and target columns for classifiers

varclf_features = ["variance_dQ_5_4"]
fullclf_features = [
    "minimum_dQ_5_4",
    "variance_dQ_5_4",
    "discharge_capacity_2",
    "diff_IR_100_2",
]
targetclf = ["cycle_550_clf"]

In [ ]:
def get_split(data, features, target, split):
    X = data.iloc[split,:].loc[:,features]
    y = data.iloc[split,:].loc[:,target]
    return X, y

def eval_model(model, data, features, target, split):
    rmse = list()
    mpe = list()
    for split in splits:
        X, y = get_split(data, features, target, split)
        pred = model.predict(X)
        rmse.append(np.sqrt(mean_squared_error(pred, y)))
        mpe.append(float(np.mean(np.abs((y - pred.reshape(-1,1))) / y * 100)))
    return rmse, mpe

def eval_classifier(model, data, features, target, splits):
    acc = list()    
    for split in splits:
        X, y = get_split(data, features, target, split)
        pred = model.predict(X)
        acc.append(accuracy_score(pred, y.values.ravel()))
    return acc

# Variance Model

In [ ]:
# Train Elastic net
x_train, y_train = get_split(df, varmod_features, targetmod, train_ind)

alphas = np.linspace(0.0001,1,30)
parameters = {
    "alpha": alphas,
    "l1_ratio": [0.01, 0.25, 0.5, 0.75, 1.]
}
enet = ElasticNet(random_state=54)
regr = GridSearchCV(enet, parameters, cv=4, iid=False)
print("Elastic Net: %s" % regr.fit(x_train, y_train).score(x_train, y_train))

"""
Because an elastic net with alpha = 0 is technically a linear regression
and elastic net produces inaccuracies with a small alpha,
we also train a linear regression model.
Linear regression performs slighty better at RMSE,
Elastic net performs slightly better at MPE.
We decide to take the linear regression scores.
"""
lin_reg = LinearRegression()
print("Linear Regression: %s" % lin_reg.fit(x_train, y_train).score(x_train, y_train))

varmod_rmse, varmod_mpe = eval_model(lin_reg, df, varmod_features, targetmod, splits)

# Discharge Model

In [ ]:
# Train Elastic net
x_train, y_train = get_split(df, dismod_features, targetmod, train_ind)

alphas = np.linspace(0.1,1,20)
parameters = {
    "alpha": alphas,
    "l1_ratio": [0.01, 0.25, 0.5, 0.75, 1.]
}
enet = ElasticNet(random_state=54)
regr = GridSearchCV(enet, parameters, cv=4, iid=False)
print("Elastic Net: %s" % regr.fit(x_train, y_train).score(x_train, y_train))

dismod_rmse, dismod_mpe = eval_model(regr, df, dismod_features, targetmod, splits)

# Full Model

In [ ]:
# Train Elastic net model
# raising the alpha minimum to 0.59 silences the convergence warnings,
# but decreases the score significantly - what's wrong here? 

x_train, y_train = get_split(df, fullmod_features, targetmod, train_ind)

alphas = np.linspace(0.001,1,20)
parameters = {
    "alpha": alphas,
    "l1_ratio": [0.001, 0.75, 1.]
}
enet = ElasticNet(random_state=54)
regr = GridSearchCV(enet, parameters, cv=4, iid=False)
print("Elastic Net: %s" % regr.fit(x_train, y_train).score(x_train, y_train))

fullmod_rmse, fullmod_mpe = eval_model(regr, df, fullmod_features, targetmod, splits)

# Evaluate all linear regression models

In [ ]:
pd.DataFrame({"Model":["Variance model", "Discharge model", "Full model"],
              "RMSE - Train": [varmod_rmse[0],dismod_rmse[0],fullmod_rmse[0]],
              "RMSE - Primary test": [varmod_rmse[1],dismod_rmse[1],fullmod_rmse[1]],
              "RMSE - Secondary test": [varmod_rmse[2],dismod_rmse[2],fullmod_rmse[2]],
              "MPE - Train": [varmod_mpe[0],dismod_mpe[0],fullmod_mpe[0]],
              "MPE - Primary test": [varmod_mpe[1],dismod_mpe[1],fullmod_mpe[1]],
              "MPE - Secondary test": [varmod_mpe[2],dismod_mpe[2],fullmod_mpe[2]]})                                                                       

# Variance Classifier

In [ ]:
# Train Logistic Regression
x_train, y_train = get_split(df, varclf_features, targetclf, train_ind)

parameters = {"C": [0.01,0.1,0.5,0.75,1]}

logreg = LogisticRegression(solver="liblinear", random_state=54)
clf = GridSearchCV(logreg, parameters, cv=4, iid=False)
print("Logreg: %s" % clf.fit(x_train, y_train.values.ravel()).score(x_train, y_train.values.ravel()))

varclf_acc = eval_classifier(clf, df, varclf_features, targetclf, splits)

# Full Classifier

In [ ]:
# Train Logistic Regression
# Why is the full classifier worse than the variance classifier?
x_train, y_train = get_split(df, fullclf_features, targetclf, train_ind)

parameters = {"C": [0.01,0.1,0.5,0.75,1]}

logreg = LogisticRegression(solver="liblinear", random_state=54)
clf = GridSearchCV(logreg, parameters, cv=4, iid=False)
print("Logreg: %s" % clf.fit(x_train, y_train.values.ravel()).score(x_train, y_train.values.ravel()))

fullclf_acc = eval_classifier(clf, df, fullclf_features, targetclf, splits)

# Evaluate all classifiers

In [ ]:
pd.DataFrame({"Classifier":["Variance classifier", "Full classifier"],
              "Acc - Train": [varclf_acc[0],fullclf_acc[0]],
              "Acc - Primary test": [varclf_acc[1],fullclf_acc[1]],
              "Acc - Secondary test": [varclf_acc[2],fullclf_acc[2]]})                                  